**Nome**: M - **NUSP**: 

# Exercício 2

## Multilayer Perceptron


Deve-se implementar uma rede multilayer perceptron e utilizá-la para resolver dois problemas (XOR e Auto-encoder).

Uma rede de perceptrons multicamada é capaz de lidar com tais tipos de problemas não linearmente separáveis.

No entanto, em uma rede multicamadas, o erro da saída recebe influência indireta da saída de todos os outros neurônios da rede. Portanto, deve-se modificar os pesos da camada de saída para as camadas anteriores, ocorrendo assim o processo de *backpropagation* do erro.

O algoritmo *backpropagation* corrige os pesos na direção de otimização da função de custo, que no caso desta implementação é (saída esperada - saída predita)

Para realizar tal otimização, calcula-se o gradiente descendente. O calculo dos gradientes foi realizado seguindo a regra delta:

$\delta_{0} = error * \dfrac{d}{dx}\varphi(S_{n})$

$\delta_{i} = delta_{i-1} \cdot W^{T}_{i} \cdot \dfrac{d}{dx}\varphi(S_{i}), \; i = 1,2,3,...,n-1$

em que $\varphi(\cdot)$ é a função de ativação, $S_{i}$ é o vetor de *output* da camada $i$, $W_{i}$ são os pesos da camada $i$


Abaixo segue a implementação da rede MLP.

In [1]:
import numpy as np
from numpy import exp, random
import os

In [2]:
# Função de ativação sigmoid
def sigmoid(x):
  return 1/(1 + np.exp(-x))

# Derivada da função sigmoid
def derivative_sigmoid(x):
  return x * (1 - x)

# Função de ativação tanh
def tanh(x):
  return np.tanh(x)

# Derivada da função de ativação tanh
def derivative_tanh(x):
  # na backpropagation o valor aqui recebido sera o de tanh(x)
  return 1 - np.power(x, 2)

# Função de ativação Rectified Linear Unit
def reLU(x):
  np.maximum(0, x)

# Derivada da função ReLU
def derivative_reLU(x):
  if x > 0:
    return 1
  else:
    return 0

def vector_error(out, y):
  # (saida esperada - saida predita)
  return y - out


class MLP:

  def __init__(self, layers=None, seed=None):
    if seed is not None:
      np.random.seed(seed)

    if layers == None:
      print('Por favor defina o tamanho das camadas com a função set_camadas().\n Ex: set_layers([tamanho_camada_entrada, tamanho_camada_escondida, tamanho_camada_saida]).')
      return
    else: # Os pesos para cada camadas possuem a dimensão (nro de neuronios da camada anterior x nro de neuronios da camada atual )
      # isso ocorre pois cada neuronio da proxima camada tera a quantidade pesos igual a quantidade de saidas/neuronios da camada anterior (todas as linhas de cada coluna)
      # enquanto teremos varios vetores de pesos para cada neuronio da camada atual (cada coluna eh um vetor peso para cada unidade do layer atual)
      # portanto temos: cada coluna possui os pesos para cada unidade da camada atual
      # 'a' esta sempre atrasado em 1 em relação a 'b'
      self.weights = [random.randn(a, b) for a, b in zip(layers[:-1], layers[1:])]
      self.bias = [random.randn(1, a) for a in layers[1:]]
      return

  def set_layers(self, layers):
    self.weights = [random.randn(a, b) for a, b in zip(layers[:-1], layers[1:])]
    self.bias = [random.randn(1, a) for a in layers[1:]]
    return

  # Calcula a saida da MLP dado os samples de entrada em X
  def forward(self, X, activation_function=sigmoid):
    # A matriz all_outputs_nn guarda todos os outputs dos layers do MLP
    all_outputs_nn = [np.array(X)]
    for weight, bias in zip(self.weights, self.bias):
      # Saida de um neuronio antes da ativação = X * W + b
      X = sigmoid(np.dot(X, weight) + bias)
      all_outputs_nn.append(X)
    return X, all_outputs_nn

  def backpropagation(self, sample, l_rate, act_func=sigmoid, derivative_act_func=derivative_sigmoid, cost_func=vector_error):
    x, y = sample
    output, all_outputs_nn = self.forward(x, act_func)
    error = cost_func(output, y)

    # Devemos ajustar os pesos e bias de acordo com o erro
    delta = [error * derivative_act_func(output)]
    for i in range(len(all_outputs_nn)-1 , 0 , -1):
      delta_aux = delta[len(delta)-1].dot(self.weights[i-1].T) * derivative_act_func(all_outputs_nn[i-1])
      delta.append(delta_aux)
    # Invertemos a ordem do array delta e ignoramos o ultimo ("primeiro") delta
    delta = delta[::-1][1:]

    # otimizacao dos pesos e calculo dos gradientes
    for i,w in enumerate(self.weights):
      if len(all_outputs_nn[i].shape) == 1:
        all_outputs_nn[i] = all_outputs_nn[i].reshape(1, len(all_outputs_nn[i]))
      w += (l_rate * np.dot(all_outputs_nn[i].T, delta[i]))

    # otimizando bias
    for i, b in enumerate(self.bias):
      b += (l_rate * delta[i])
    return

  def train(self, samples, l_rate=0.5, error_lim=1e-3, cost_func=vector_error):
    error_list = []
    epoch = 0

    while True:
      epoch_error = []

      for sample in samples:
        x, y = sample
        output, _ = self.forward(x)
        epoch_error.append(cost_func(output, y))
        self.backpropagation(sample, l_rate=l_rate)

      mse = np.sum(np.square(epoch_error))/len(epoch_error)
      error_list.append(mse)

      if epoch % 5000 == 0:
        print('Epoch: {:05d} | MSE: {:.8f}'.format(epoch, mse))

      if mse < error_lim:
        break

      epoch += 1

    print('Epoch: {:05d} | MSE: {:.8f}'.format(epoch, mse))
    return error_list

## Primeiro problema (XOR)


O primeiro problema é o OU EXCLUSIVO (XOR).
Dado inputs $x_1$ e $x_2$, temos o seguintes casos:
- $x_1 = 0,\ x_2 = 0 \implies y = 0$
- $x_1 = 0,\ x_2 = 1 \implies y = 1$
- $x_1 = 1,\ x_2 = 0 \implies y = 1$
- $x_1 = 1,\ x_2 = 1 \implies y = 0$

A rede MLP deve ser treinada para ser capaz de predizer corretamente as saídas esperadas.

A arquitetura da rede é a seguinte:
- Camada de entrada: Número de inputs ($x_1$ e $x_2$),
- Camada oculta: 4 neurônios
- Camada de saída: 1 neurônio

In [3]:
xor = [([0, 0], [0]), ([0, 1], [1]), ([1, 0], [1]), ([1, 1], [0])]

In [4]:
mlp_xor = MLP(layers=[len(xor[0]), 4, 1])

In [5]:
error_epoch_xor = mlp_xor.train(samples=xor, l_rate=0.5, error_lim=1e-3)

Epoch: 00000 | MSE: 0.29679574
Epoch: 02554 | MSE: 0.00099975


In [6]:
for x, _ in xor:
  output, _ = mlp_xor.forward(x)
  print(f'{x} => {np.round(output)}')

[0, 0] => [[0.]]
[0, 1] => [[1.]]
[1, 0] => [[1.]]
[1, 1] => [[0.]]


## Segundo problema (Auto-encoder)

Um auto-encoder é uma rede na qual a saída é igual a entrada. Neste tipo de arquitetura, busca-se realizar um mapeamento dos atributos de entrada em uma camada intermediária com menos neurônios do que o número de entradas (no caso deste exercício, a camada intermediária de mapeamento contém $\log_{2}N$ neurônios, com $N$ sendo o tamanho da entrada).

### Id8

Resolvendo o problema do autoassociador para uma matriz identidade $I_{8 \times 8}$.

In [7]:
Id8 = [(np.identity(8).flatten(), np.identity(8).flatten())]
#print(Id8)
x, y = Id8[0]

In [8]:
mlp_id8 = MLP(layers=[len(x), int(np.log2(len(x))), len(x)])

In [9]:
error_epoch_id8 = mlp_id8.train(samples=Id8, l_rate=0.5, error_lim=1e-3)

Epoch: 00000 | MSE: 20.98008839
Epoch: 05000 | MSE: 0.00306995
Epoch: 10000 | MSE: 0.00154756
Epoch: 15000 | MSE: 0.00103812
Epoch: 15581 | MSE: 0.00099998


In [10]:
for x, _ in Id8:
  output, _ = mlp_id8.forward(x)
  print(f'Saida esperada:\n {x.reshape(8,8)}\n Saida da MLP (depois de arredondado):\n {np.round(output.reshape(8,8))}')

Saida esperada:
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
 Saida da MLP (depois de arredondado):
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


### Id15

Resolvendo o problema do autoassociador para uma matriz identidade $I_{15 \times 15}$.

In [11]:
Id15 = [(np.identity(15).flatten(), np.identity(15).flatten())]
#print(Id15)
x, y = Id15[0]

In [12]:
mlp_id15 = MLP(layers=[len(x), int(np.log2(len(x))), len(x)])

In [13]:
error_epoch_id15 = mlp_id15.train(samples=Id15, l_rate=0.5, error_lim=1e-3)

Epoch: 00000 | MSE: 84.51331461
Epoch: 05000 | MSE: 0.00608632
Epoch: 10000 | MSE: 0.00308023
Epoch: 15000 | MSE: 0.00206910
Epoch: 20000 | MSE: 0.00155945
Epoch: 25000 | MSE: 0.00125185
Epoch: 30000 | MSE: 0.00104588
Epoch: 31396 | MSE: 0.00099998


In [14]:
for x, _ in Id15:
  output, _ = mlp_id15.forward(x)
  print(f'Saida esperada:\n {x.reshape(15,15)}\n Saida da MLP (depois de arredondado):\n {np.round(output.reshape(15,15))}')

Saida esperada:
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
 Saida da MLP (depois de arredondado):
 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0